## Classification and dimensionality reduction

In [1]:
LOCAL = True

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext skip_cell

In [3]:
%%skip $LOCAL
#Mounting the drive

import zipfile
from google.colab import drive

drive.mount('/content/drive/')

In [4]:
%%skip $LOCAL

!cp -a "/content/drive/My Drive/triplets/" .

Setting up tensorboard for PyTorch in Colab

In [5]:
%%skip $LOCAL

!pip install -q tf-nightly-2.0-preview
%load_ext tensorboard
import os
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

# Imports

In [6]:
import copy
import random
import time
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import torch
from torch import optim
from torch import nn
from torch.utils.data import random_split
# from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torchvision.datasets import SVHN
from torchvision.models import resnet18
from typing import Union
from PIL import Image

In [7]:
from triplets.datasets import TripletSVHN
from triplets.losses import TripletLoss, TripletSoftLoss
from triplets.metrics import mean_average_precision
from triplets.nets import TripletNet
from triplets.train import train
from triplets.transformers import FeatureExtractor

ModuleNotFoundError: No module named 'triplets'

In [ ]:
n_features = 512
n_classes = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
SEED = 100
validation_split = 0.2
shuffle_dataset = True

In [ ]:
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
dataset = SVHN(root = 'data/', download=True, split='train')
train_size = int(0.8*len(dataset))
valid_size = len(dataset) - train_size
dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size])
dataset_test = SVHN(root = 'data/', download=True, split='test');

# Training model with triplet loss

In [ ]:
batch_size = 32
num_triplets = 1
epochs = 20

In [ ]:
model = resnet18(pretrained=True)
model.eval();

Defining extractor using custom class to extract features from last cnn pretrained layer.

In [ ]:
extractor = FeatureExtractor(model=model, n_remove_layers=1, n_features=n_features, device=device)
extracted_resnet = extractor.prepare_model()

Freeze all the layers besides two last ones (pooling and two convolutional blocks)

In [ ]:
for idx, child in enumerate(extracted_resnet.children()):
    if idx < 7:
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
preprocess = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),                
 transforms.ToTensor(),                     
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])

Defining triplet datasets and dataloaders

In [ ]:
triplet_train= TripletSVHN(dataset, dataset_train.indices, dataset_valid.indices,
                            preprocess, 'train', SEED)
triplet_valid = TripletSVHN(dataset, dataset_train.indices, dataset_valid.indices,
                            preprocess, 'val', SEED)

In [ ]:
dataloader_train = torch.utils.data.DataLoader(triplet_train, batch_size=batch_size)
dataloader_valid = torch.utils.data.DataLoader(triplet_valid, batch_size=batch_size)
dataloaders = {'train': dataloader_train, 'val': dataloader_valid}

In [ ]:
model = TripletNet(extracted_resnet)
criterion = TripletSoftLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
test_sample = next(iter(dataloader_train))

In [ ]:
test_out = model(*test_sample[0])

In [ ]:
criterion(*test_out)

In [ ]:
%%skip $LOCAL

writer = SummaryWriter()

In [ ]:
train(model, dataloaders, criterion, optimizer, scheduler, epochs, device)

#TODO : Probably combine notebooks (using knn on learned embeddings with margin hyperparameter loss optimisation) - apply negative mining techniques, fix TripletDataset